# GPT-3.5-Turbo on GSM8K

In [65]:
import re
import time
import os
import numpy as np
import pickle

import openai
from tqdm import tqdm
from datasets import load_dataset


from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append("../..")

from llms import call_llm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open('data/multi_gsm8k.pickle', 'rb') as handle:
    gsm8k = pickle.load(handle)

In [15]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [16]:
gsm8k_test = gsm8k['test']
gsm8k_test_es = gsm8k['test_es_gt']


In [17]:
prompt_original_es = open('../gsm8k/prompts/prompt_original_es.txt').read()

print(prompt_original_es)

Pregunta: Hay 15 árboles en el bosque. Los trabajadores de la arboleda plantarán árboles en la arboleda hoy. Después de que hayan terminado, habrá 21 árboles. ¿Cuántos árboles plantaron hoy los trabajadores de la arboleda?
Pensemos paso a paso
Hay 15 árboles originalmente.
Luego hubo 21 árboles después de que se plantaron algunos más.
Entonces debe haber habido 21 - 15 = 6.
la respuesta es 6

Pregunta: Si hay 3 autos en el estacionamiento y llegan 2 autos más, ¿cuántos autos hay en el estacionamiento?
Pensemos paso a paso
Originalmente hay 3 coches.
Llegan 2 carros más.
3 + 2 = 5.
la respuesta es 5

Pregunta: Leah tenía 32 chocolates y su hermana tenía 42. Si se comieron 35, ¿cuántos pedazos les quedaron en total?
Pensemos paso a paso
Originalmente, Leah tenía 32 chocolates.
Su hermana tenía 42.
Así que en total tenían 32 + 42 = 74.
Después de comer 35, tenían 74 - 35 = 39.
La respuesta es 39.

Pregunta: Jason tenía 20 piruletas. Le dio a Denny unas piruletas. Ahora Jason tiene 12 piru

In [5]:
prompt_original = open('../gsm8k/prompts/prompt_original.txt').read()

print(prompt_original)

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Let's think step by step
There are 15 trees originally.
Then there were 21 trees after some more were planted.
So there must have been 21 - 15 = 6.
The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Let's think step by step
There are originally 3 cars.
2 more cars arrive.
3 + 2 = 5.
The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Let's think step by step
Originally, Leah had 32 chocolates.
Her sister had 42.
So in total they had 32 + 42 = 74.
After eating 35, they had 74 - 35 = 39.
The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Let's

In [25]:
prompt_q = prompt_original + '\nQuestion: ' + gsm8k_test[2]['question'] + '\n'

print(prompt_q)

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Let's think step by step
There are 15 trees originally.
Then there were 21 trees after some more were planted.
So there must have been 21 - 15 = 6.
The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Let's think step by step
There are originally 3 cars.
2 more cars arrive.
3 + 2 = 5.
The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Let's think step by step
Originally, Leah had 32 chocolates.
Her sister had 42.
So in total they had 32 + 42 = 74.
After eating 35, they had 74 - 35 = 39.
The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Let's

In [26]:
prompt_q_es = prompt_original_es + '\nPregunta: ' + gsm8k_test_es[2]['question'] + '\n'

print(prompt_q_es)

Pregunta: Hay 15 árboles en el bosque. Los trabajadores de la arboleda plantarán árboles en la arboleda hoy. Después de que hayan terminado, habrá 21 árboles. ¿Cuántos árboles plantaron hoy los trabajadores de la arboleda?
Pensemos paso a paso
Hay 15 árboles originalmente.
Luego hubo 21 árboles después de que se plantaron algunos más.
Entonces debe haber habido 21 - 15 = 6.
la respuesta es 6

Pregunta: Si hay 3 autos en el estacionamiento y llegan 2 autos más, ¿cuántos autos hay en el estacionamiento?
Pensemos paso a paso
Originalmente hay 3 coches.
Llegan 2 carros más.
3 + 2 = 5.
la respuesta es 5

Pregunta: Leah tenía 32 chocolates y su hermana tenía 42. Si se comieron 35, ¿cuántos pedazos les quedaron en total?
Pensemos paso a paso
Originalmente, Leah tenía 32 chocolates.
Su hermana tenía 42.
Así que en total tenían 32 + 42 = 74.
Después de comer 35, tenían 74 - 35 = 39.
La respuesta es 39.

Pregunta: Jason tenía 20 piruletas. Le dio a Denny unas piruletas. Ahora Jason tiene 12 piru

In [31]:
response_es = call_llm(
  model="text-bison@001",
  prompt="Sigue los ejemplos dados y responde la pregunta. " + prompt_q_es
)
print(response_es)

In [27]:
response = call_llm(
  model="chat-bison@001",
  prompt="Follow the given examples and answer the question. " + prompt_q
)
print(response)

Let's think step by step
Josh bought the house for $80,000.
He put in $50,000 in repairs.
So the total cost of the house is 80,000 + 50,000 = $130,000.
The value of the house increased by 150%.
So the value of the house is 130,000 * 150 / 100 = $195,000.
So Josh made a profit of 195,000 - 130,000 = $65,000.
The answer is 65,000.


In [28]:
response = call_llm(
  model="text-bison@001",
  prompt="Follow the given examples and answer the question. " + prompt_q
)
print(response)

Let's think step by step
The house was worth $80,000 and he put in $50,000 in repairs, so the total cost was $80,000 + $50,000 = $130,000.  The house increased in value by 150%, so the new value is $130,000 * 1.5 = $195,000.  So he made a profit of $195,000 - $130,000 = $65,000.
The answer is 65000.


In [13]:
response = call_llm(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

print(response[0])

Let's think step by step
Janet's ducks lay 16 eggs per day.
She eats 3 eggs every morning and uses 4 for baking. So, she uses 3 + 4 = 7 eggs every day.
Therefore, she has 16 - 7 = 9 eggs left for selling.
At $2 per egg, she makes 9 x 2 = 18 dollars every day.
The answer is 18 dollars.


## Test answer

In [10]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

response_finish = {
    'en': 'answer is',
    'es': 'respuesta es',
    'fr': 'réponse est',
    'pt': 'resposta é',
    'it': 'risposta è'
}
def extract_ans(ans_model, lang= ''):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if(response_finish[lang] in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

## Run

In [60]:

def run(file_output_run, test_run, prompt_run, question_run, model_run, instruction_run, lang):
    i = 0
    responses = []
    with open('outputs/' + file_output_run, 'w') as fd:
        gsm8k_test = gsm8k[test_run]
        for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                                total=len(gsm8k_test['question'])):
            
            prompt_path = f"../gsm8k/prompts/{prompt_run}"
            prompt = open(prompt_path).read()
            prompt_q = f"{prompt}\n{question_run}{q}\n" 
            #print(prompt_q)
            if model_run.startswith('gpt-3.5-turbo'):
                response = call_llm(
                    model=model_run,
                    parameters={
                        'messages': [
                                {"role": "system", "content": instruction_run},
                                {"role": "user", "content": prompt_q},
                            ],
                        'temperature': 0
                        }
                    )
            
            if model_run.startswith('text-bison@'):
                response = call_llm(
                    model=model_run,
                    parameters= {
                        'prompt': f"""{instruction_run}""" + "\n" + prompt_q,
                        'temperature': 0
                        }
                    
                    )
            
            if model_run.startswith('chat-bison@'):
                response = call_llm(
                    model=model_run,
                    parameters= {
                        'message': f"""{instruction_run}""" + "\n" + prompt_q,
                        'temperature': 0
                        }
                    
                    )
            
            responses.append(response[0])
            ans_model = response[0]
            ans_, residual = extract_ans(ans_model, lang)
                
            fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
            i += 1
            #if(i == 2): break
        
    _, _, _ = parse_pred_ans('outputs/' + file_output_run)
    file_responses = open("outputs/responses" + file_output_run[:-3] + 'pickle', 'wb')
    pickle.dump(responses, file_responses)
    file_responses.close()
    #return responses


# GPT-3.5 Evaluations

## Baseline Prompt Greedy Decoding, Acc 74.98 0.7536

In [12]:
prompt = 'prompt_original.txt'
test = 'test'
question = "Question: "
instruction = "Follow the given examples and answer the question."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'en'
run(file_output, test, prompt, question, model, instruction, lang)

  0%|          | 0/1319 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [19]:
prompt = 'prompt_original.txt'
test = 'test'
question = "Question: "
instruction = "Follow the given examples and answer the question."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'en'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:16:36<00:00,  6.21s/it]    

num_q 1319 correct 994 ratio 0.7536


TypeError: write() argument must be str, not bytes

## Baseline Prompt Greedy Decoding Spanish, Acc 69.52 0.7036


In [23]:
prompt = 'prompt_original_es.txt'
test = 'test_es_gt'
question = "Pregunta: "
instruction = "Sigue los ejemplos dados y responde la pregunta."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'es'
responses = run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:38:09<00:00,  7.19s/it]   

num_q 1319 correct 928 ratio 0.7036


## Baseline Prompt Greedy Decoding Portuguese, Acc 63.53 0.6323


In [30]:
prompt = 'prompt_original_pt.txt'
test = 'test_pt_gt'
question = "Pergunta: "
instruction = "Siga os exemplos dados e responda à questão."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'pt'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:35:30<00:00,  7.07s/it]  

num_q 1319 correct 834 ratio 0.6323


[<OpenAIObject chat.completion id=chatcmpl-7SEDuHru8JZ8qh61BK56PCz3t5dWh at 0x142e7c3b0> JSON: {
   "id": "chatcmpl-7SEDuHru8JZ8qh61BK56PCz3t5dWh",
   "object": "chat.completion",
   "created": 1686962214,
   "model": "gpt-3.5-turbo-0301",
   "usage": {
     "prompt_tokens": 983,
     "completion_tokens": 127,
     "total_tokens": 1110
   },
   "choices": [
     {
       "message": {
         "role": "assistant",
         "content": "Vamos pensar passo a passo\nOs patos de Janet p\u00f5em 16 ovos por dia.\nEla come 3 ovos todas as manh\u00e3s, ent\u00e3o sobram 16 - 3 = 13 ovos.\nEla usa 4 ovos para fazer muffins para seus amigos, ent\u00e3o sobram 13 - 4 = 9 ovos.\nEla vende os 9 ovos restantes no mercado dos fazendeiros por $ 2 cada, ent\u00e3o ela ganha 9 x 2 = $ 18 por dia.\nA resposta \u00e9 $ 18."
       },
       "finish_reason": "stop",
       "index": 0
     }
   ]
 },
 <OpenAIObject chat.completion id=chatcmpl-7SEDzkc7bhV20zHL6SlIwpVdZAMV1 at 0x1432af8d0> JSON: {
   "id": "ch

## Baseline Prompt Greedy Decoding French, Acc 68.92 0.6846


In [31]:
prompt = 'prompt_original_fr.txt'
test = 'test_fr_gt'
question = "Question: "
instruction = "Suivez les exemples donnés et répondez à la question."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'fr'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [3:14:05<00:00,  8.83s/it]   

num_q 1319 correct 903 ratio 0.6846


[<OpenAIObject chat.completion id=chatcmpl-7SRbYZVFwuONuNfvHdW5gQXej3Dut at 0x1188df290> JSON: {
   "id": "chatcmpl-7SRbYZVFwuONuNfvHdW5gQXej3Dut",
   "object": "chat.completion",
   "created": 1687013652,
   "model": "gpt-3.5-turbo-0301",
   "usage": {
     "prompt_tokens": 1041,
     "completion_tokens": 109,
     "total_tokens": 1150
   },
   "choices": [
     {
       "message": {
         "role": "assistant",
         "content": "R\u00e9fl\u00e9chissons \u00e9tape par \u00e9tape\nLes canards de Janet pondent 16 \u0153ufs par jour.\nElle en mange 3 pour le petit-d\u00e9jeuner et utilise 4 pour faire des muffins, donc elle en utilise 7 au total.\nElle en vend donc 16 - 7 = 9 chaque jour.\nElle gagne 2 $ par \u0153uf vendu, donc elle gagne 9 x 2 = 18 $ chaque jour.\nLa r\u00e9ponse est 18 $."
       },
       "finish_reason": "stop",
       "index": 0
     }
   ]
 },
 <OpenAIObject chat.completion id=chatcmpl-7SRbdJX1k5uRMLKVzghSSYp8WcdmV at 0x147156d90> JSON: {
   "id": "chatcmpl-7S

## Baseline Prompt Greedy Decoding Italian, Acc 66.94 0.6725


In [33]:
prompt = 'prompt_original_it.txt'
test = 'test_it_gt'
question = "Domanda: "
instruction = "Segui gli esempi proposti e rispondi alla domanda."
model = "gpt-3.5-turbo"
file_output = f"{test}_{model}_{prompt}"
lang = 'it'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:52:30<00:00,  7.85s/it]   

num_q 1319 correct 887 ratio 0.6725


# PALM-2 Text-Bison Evaluations

0.6513

In [12]:
prompt = 'prompt_original.txt'
test = 'test'
question = "Question: "
instruction = "Follow the given examples and answer the question."
model = "text-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'en'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:50:59<00:00,  7.78s/it]  

num_q 1319 correct 859 ratio 0.6513


0.6202

In [42]:
prompt = 'prompt_original_es.txt'
test = 'test_es_gt'
question = "Pregunta: "
instruction = "Sigue los ejemplos dados y responde la pregunta."
model = "text-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'es'
responses = run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:49:21<00:00,  7.70s/it]  

num_q 1319 correct 818 ratio 0.6202


In [ ]:
prompt = 'prompt_original_pt.txt'
test = 'test_pt_gt'
question = "Pergunta: "
instruction = "Siga os exemplos dados e responda à questão."
model = "text-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'pt'
run(file_output, test, prompt, question, model, instruction, lang)

In [ ]:
prompt = 'prompt_original_fr.txt'
test = 'test_fr_gt'
question = "Question: "
instruction = "Suivez les exemples donnés et répondez à la question."
model = "text-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'fr'
run(file_output, test, prompt, question, model, instruction, lang)

In [ ]:
prompt = 'prompt_original_it.txt'
test = 'test_it_gt'
question = "Domanda: "
instruction = "Segui gli esempi proposti e rispondi alla domanda."
model = "text-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'it'
run(file_output, test, prompt, question, model, instruction, lang)

# PALM-2 Chat-Bison Evaluations


0.6399


In [38]:
prompt = 'prompt_original.txt'
test = 'test'
question = "Question: "
instruction = "Follow the given examples and answer the question."
model = "chat-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'en'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [3:07:54<00:00,  8.55s/it]  

num_q 1319 correct 844 ratio 0.6399


0.6050

In [39]:
prompt = 'prompt_original_es.txt'
test = 'test_es_gt'
question = "Pregunta: "
instruction = "Sigue los ejemplos dados y responde la pregunta."
model = "chat-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'es'
responses = run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [3:08:39<00:00,  8.58s/it]  

num_q 1319 correct 798 ratio 0.6050


In [40]:
prompt = 'prompt_original_pt.txt'
test = 'test_pt_gt'
question = "Pergunta: "
instruction = "Siga os exemplos dados e responda à questão."
model = "chat-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'pt'
run(file_output, test, prompt, question, model, instruction, lang)

 62%|██████▏   | 816/1319 [7:25:10<4:34:24, 32.73s/it]


KeyboardInterrupt: 

In [41]:
prompt = 'prompt_original_fr.txt'
test = 'test_fr_gt'
question = "Question: "
instruction = "Suivez les exemples donnés et répondez à la question."
model = "chat-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'fr'
run(file_output, test, prompt, question, model, instruction, lang)

100%|██████████| 1319/1319 [2:42:11<00:00,  7.38s/it]  

num_q 1319 correct 0 ratio 0.0000


In [ ]:
prompt = 'prompt_original_it.txt'
test = 'test_it_gt'
question = "Domanda: "
instruction = "Segui gli esempi proposti e rispondi alla domanda."
model = "chat-bison@001"

file_output = f"{test}_{model}_{prompt}"
lang = 'it'
run(file_output, test, prompt, question, model, instruction, lang)